## grab all MERRA2 fields for the bounding box +/- 5 degrees and write out 

To help us composite gridded quantities by distance from the CWV=55mm boundary contour, with a `groupby().mean()` operation, we need to construct a new distance field on the MERRA2 grid.

In [8]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import geopandas as gp

from shapely.geometry import Point
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

import warnings
warnings.filterwarnings('ignore')

# Open MERRA2 files and combine all the wanted fields 
### (2D hourly collections on goldsmr4) 

In [15]:
d2diag = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXINT')
d2flux = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXFLX')
d2slv = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXSLV')

var_diag = ['dqvdt_phy','dqvdt_dyn','dqvdt_ana','uflxqv','vflxqv','swnettoa']
var_flux = ['eflux','evap','hflux','preccon','precanv','prectot','prevtot','prectotcorr']
var_slv = [ 't2m','ts','t850','t500','q850','q500','slp','tqv','tqi','tql',  \
           'disph','tropt' ] # dummy ones to re-use as distance and direction

d2super = d2diag[ var_diag ].merge(d2flux[ var_flux ]).merge(d2slv[ var_slv ])\
        .rename({"disph":"distance"}).rename({"tropt":"dir_from_centroid"})


## Read all lakes in 2014-2018 
#### lakes dataframe was "improved" in LakeCaseStudy.ipynb with firsttime added

In [16]:
df = pd.read_csv('ccvls_stats_2014-2018.improved.csv')
# df

In [17]:
equatorcases = df[ abs(df.coastlat) < 10 ] # 162 of them exceeding 1 day 
eq7cases = equatorcases[ equatorcases.dur_days > 6 ]  # 39 of them exceeding a week (more than 6 in days) 
eq7cases.maxarea.size

39

In [ ]:
for icase in range(len(eq7cases)): 

# Continue from where it last crashed/ended:
    if(icase < 8): 
        continue

# Select case and move forward
    case = eq7cases.iloc[icase]
    filename = case.filename
    print('case ',filename, icase)

    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = d2super.sel( lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                  lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                  time=slice(gdf.time.min(),gdf.time.max())             )

    # Put a dummy distance field, initially large so min operator will replace it later 
    d2.distance.values = d2.distance.values*0 +999.
    d2['distance'].attrs['long_name'] = 'distance outside nearest lake (negative for interiors)'
    d2['distance'].attrs['units'] = 'degrees lat/lon'
    d2['dir_from_centroid'].attrs['long_name'] = 'navigation angle from centroid of nearest lake'
    d2['dir_from_centroid'].attrs['units'] = 'nav.degrees'

# Make an array of Points that are the gridpoints or MERRA2
    lat2d = d2.lat.values[:,None]   + d2.lon.values*0
    lon2d = d2.lat.values[:,None]*0 + d2.lon.values
    points = gp.GeoSeries( [Point(x, y) for x, y in zip(lon2d.ravel(),lat2d.ravel()) ] )\
            .set_crs(epsg = "4326", inplace = True)
    
    for i in range(gdf.time.size):     # the number of rows in the geoseries (polygons), not # of times
                
# Compute distance and direction fields:
# DIST: must compute distance to polygon BOUNDARY to get nonzero values inside it 
        dist = points.distance(gdf.geometry[i].boundary).values.reshape(len(d2.lat),len(d2.lon))
        isin = points.within(gdf.geometry[i]).values.reshape(len(d2.lat),len(d2.lon))
        dist *= (-2)*(isin-0.5)  # make SIGNED distance from boundary, positive is exterior 

# DIR: use geodesic method imported above. 
# Direction to centroid of WHOLE LAKE (multi-polygon in general) at this time
        agg_lake_at_time = gdf.geometry[ np.where(gdf.time == gdf.time[i])[0] ].unary_union
        centlon = agg_lake_at_time.centroid.x
        centlat = agg_lake_at_time.centroid.y
    
    #Careful, use j not i in stupid explicit loop, since geodesic takes scalar only 
        dir_to = []
        for j in range(len(lon2d.ravel())): 
            fwd_az,back_az,d = geodesic.inv(lon2d.ravel()[j], lat2d.ravel()[j], [centlon], [centlat])
            dir_to.append(fwd_az+180)
        dir_to = np.array(dir_to).reshape(len(d2.lat),len(d2.lon))
    
# if dist is less than previous distance in the dataset, replace that previous
        oldd = d2.sel(time = gdf.time[i],method='nearest').distance
        mindist = np.minimum(dist,oldd)
    # closer = np.where(dist < oldd) # a spatial index of where, couldn't make it work
    
# Here's a tricky thing, assigning values to the dataset - need to use index instead of sel
# sel cannot be used for assignment of values, only extraction of them. 
# idx of the time in d2 corresponding to this geometry object in the gdf
        idx = d2.indexes["time"].get_indexer([gdf.time[i]],method='nearest')[0]

# Assign the new mindist field at this time, & dir where (dist < oldd) for this object
        d2["distance"][idx] = mindist
    
# Assigning direction is even trickier. I can't seem to use "closer" 
# to reassign direction only in places where dist < oldd, such that 
# the direction field would be from the centroid of the CLOSEST polgon. 
# Instead, let's just assign direction from the centroid of the whole 
# unary union of all the individual polygons 

        d2["dir_from_centroid"][idx] = dir_to

#  these 1-hour averages are 30 minutes offset from tqv, argh. 
# just manually spray the mindist values to the next earlier time level too 
        try: 
            d2["distance"][idx-1] = mindist
            d2["dir_from_centroid"][idx-1] = dir_to

        except: 
            print('end of loop issue (first time in time-backward sequence over geometries)')

        print('Writing outputs')
        d2.to_netcdf('../../Library/CloudStorage/Box-Box/VaporLakes/data/LAKEBYLAKE/'+\
             filename[0:-8]+'.M2_all2Dfields.nc')
    

In [ ]:
!date

# Consider making IDV bundle for it 
### by relacing filename inside .xidv of a template bundle

In [21]:
!ls ../../Library/CloudStorage/Box-Box/VaporLakes/data/LAKEBYLAKE/

2017_05_06_20_lat3p790S.M2_all2Dfields.nc
2017_05_08_05_lat1p160N.M2_all2Dfields.nc
2017_05_09_01_lat3p355S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.IMERG_30minute.nc
2017_05_28_11_lat5p291S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.M2_all2Dfields_myfirst.nc
2017_05_28_11_lat5p291S.M2_composite.nc
2017_12_01_11_lat9p845S.M2_all2Dfields.nc
2018_03_19_12_lat9p318S.M2_all2Dfields.nc
2018_10_22_15_lat9p781N.M2_all2Dfields.nc
2018_11_16_21_lat8p412S.M2_all2Dfields.nc
2018_11_20_11_lat4p616S.M2_all2Dfields.nc
direction_field_wrong_fixable


In [32]:
# inplace replacement works 

!cp ../../Library/CloudStorage/Box-Box/VaporLakes/Landfalling_Lake_Event_Displays.xidv test.txt

!sed -i '' "s/2017_05_28_11_lat5p291S.M2_all2Dfields_myfirst/2018_11_20_11_lat4p616S.M2_all2Dfields.nc]/g" test.txt

In [33]:
!grep Box test.txt

                                <string><![CDATA[/Users/brianmapes/Library/CloudStorage/Box-Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>
                        <string><![CDATA[/Users/bem/Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>
                                        <string><![CDATA[/Users/bem/Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>


In [ ]:
template = '../../Library/CloudStorage/Box-Box/VaporLakes/Landfalling_Lake_Event_Displays.xidv'